# Output Parser Patterns with Amazon Nova

This notebook demonstrates parsing and structuring LLM outputs using LangChain parsers.

## Setup

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [2]:
from langchain_amazon_nova import ChatAmazonNova
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
    PydanticOutputParser,
)
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

# Initialize the model
llm = ChatAmazonNova(model="nova-pro-v1", temperature=0)

## 1. String Parser

The simplest parser - converts AIMessage to string.

In [3]:
str_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")
chain = prompt | llm | str_parser

result = chain.invoke({"country": "Japan"})
print(f"Result: {result}")
print(f"Type: {type(result)}")

Result: The capital of Japan is Tokyo. Tokyo, officially known as Tokyo Metropolis, is one of the 47 prefectures of Japan. It is located in the Kantō region on the southeastern side of the main island of Honshu. Tokyo is the political, economic, and cultural center of Japan and is one of the most populous metropolitan areas in the world.

### Key Points about Tokyo:

#### 1. **Historical Background:**
   - Originally a small fishing village named Edo, Tokyo grew into a major city during the Edo period (1603–1868) when it served as the seat of the Tokugawa shogunate.
   - After the Meiji Restoration in 1868, the city was renamed Tokyo, meaning "Eastern Capital," and became the new seat of the Emperor.

#### 2. **Population:**
   - Tokyo is home to over 13 million people within its city limits. The Greater Tokyo Area, which includes neighboring prefectures, has a population of over 37 million, making it the most populous metropolitan area in the world.

#### 3. **Economy:**
   - Tokyo is

## 2. Comma-Separated List Parser

Parse comma-separated values into a list.

In [4]:
list_parser = CommaSeparatedListOutputParser()

prompt = ChatPromptTemplate.from_template("List 5 {category}.\n{format_instructions}")

chain = prompt | llm | list_parser
result = chain.invoke(
    {
        "category": "programming languages",
        "format_instructions": list_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"First item: {result[0]}")

Result: ['Python', 'Java', 'JavaScript', 'C++', 'Ruby']
Type: <class 'list'>
First item: Python


## 3. JSON Parser

Parse JSON output into Python dictionaries.

In [5]:
json_parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_template(
    "Provide information about {topic} in JSON format with keys: name, description, year_created.\n{format_instructions}"
)

chain = prompt | llm | json_parser
result = chain.invoke(
    {
        "topic": "Python programming language",
        "format_instructions": json_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"Name: {result.get('name')}")

Result: {'name': 'Python', 'description': 'Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python has a vast standard library and an extensive ecosystem of third-party packages, making it suitable for a wide range of applications, from web development and data analysis to artificial intelligence and scientific computing.', 'year_created': 1991}
Type: <class 'dict'>
Name: Python


## 4. Pydantic Parser

Parse into strongly-typed Pydantic models.

In [6]:
class Person(BaseModel):
    """Person information."""

    name: str = Field(description="Person's name")
    age: int = Field(description="Person's age")
    occupation: str = Field(description="Person's occupation")


pydantic_parser = PydanticOutputParser(pydantic_object=Person)

prompt = ChatPromptTemplate.from_template(
    "Generate a fictional person who is a {occupation}.\n{format_instructions}"
)

chain = prompt | llm | pydantic_parser
result = chain.invoke(
    {
        "occupation": "software engineer",
        "format_instructions": pydantic_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Occupation: {result.occupation}")

Result: name='Alex Johnson' age=30 occupation='Software Engineer'
Type: <class '__main__.Person'>
Name: Alex Johnson
Age: 30
Occupation: Software Engineer


## 5. Complex Pydantic Model

Parse into nested structures with multiple fields.

In [7]:
class StoryAnalysis(BaseModel):
    """Story analysis."""

    main_characters: List[str] = Field(description="Main characters in the story")
    setting: str = Field(description="Where the story takes place")
    theme: str = Field(description="Primary theme")


story_parser = PydanticOutputParser(pydantic_object=StoryAnalysis)

prompt = ChatPromptTemplate.from_template(
    "Analyze this story:\n{story}\n\n{format_instructions}"
)

chain = prompt | llm | story_parser
result = chain.invoke(
    {
        "story": "Alice went to Wonderland and had tea with the Mad Hatter. It was a strange adventure about growing up.",
        "format_instructions": story_parser.get_format_instructions(),
    }
)

print(f"Characters: {result.main_characters}")
print(f"Setting: {result.setting}")
print(f"Theme: {result.theme}")

Characters: ['Alice', 'Mad Hatter']
Setting: Wonderland
Theme: A strange adventure about growing up


## 6. Chaining Multiple Parsers

Use different parsers in a workflow.

In [8]:
# Step 1: Generate list of topics
list_prompt = ChatPromptTemplate.from_template(
    "List 3 {category}.\n{format_instructions}"
)
list_parser = CommaSeparatedListOutputParser()
list_chain = list_prompt | llm | list_parser

topics = list_chain.invoke(
    {
        "category": "animals",
        "format_instructions": list_parser.get_format_instructions(),
    }
)

print(f"Generated topics: {topics}\n")

# Step 2: Get JSON info for first topic
json_prompt = ChatPromptTemplate.from_template(
    "Provide brief info about {topic} as JSON with keys: name, habitat, diet.\n{format_instructions}"
)
json_parser = JsonOutputParser()
json_chain = json_prompt | llm | json_parser

info = json_chain.invoke(
    {"topic": topics[0], "format_instructions": json_parser.get_format_instructions()}
)

print(f"Info about {topics[0]}:")
print(f"  Habitat: {info.get('habitat')}")
print(f"  Diet: {info.get('diet')}")

Generated topics: ['Dog', 'cat', 'elephant']

Info about Dog:
  Habitat: Domesticated, lives with humans in various environments including homes, urban areas, and rural settings.
  Diet: Omnivorous; commonly fed commercial dog food, which includes meat, vegetables, and grains. Can also consume a variety of human foods (in moderation) and specialized diets depending on health needs.


## Summary

**Output Parser Patterns:**

| Parser | Output Type | Use Case |
|--------|-------------|----------|
| `StrOutputParser` | `str` | Simple text extraction |
| `CommaSeparatedListOutputParser` | `List[str]` | List generation |
| `JsonOutputParser` | `dict` | Flexible structured data |
| `PydanticOutputParser` | `BaseModel` | Type-safe structured data |

**Key Benefits:**
- **Type Safety**: Pydantic models provide validation
- **Format Instructions**: Parsers generate prompts automatically
- **Error Handling**: Built-in validation and parsing errors
- **Composability**: Mix parsers in chains
- **Documentation**: Models serve as data contracts

**When to Use:**
- **String Parser**: Simple text responses
- **List Parser**: Generating options, enumerations
- **JSON Parser**: Flexible data without strict schema
- **Pydantic Parser**: APIs, databases, strict validation